## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["ELEVEN_API_KEY"] = os.getenv("ELEVEN_API_KEY")
os.environ["AZURE_COGS_KEY"] = os.getenv("AZURE_COGS_KEY")
os.environ["AZURE_COGS_ENDPOINT"] = os.getenv("AZURE_COGS_ENDPOINT")
os.environ["AZURE_COGS_REGION"] = os.getenv("AZURE_COGS_REGION")

### LCEL:memory添加方式
### LCEL:memory add mode
<hr>

In [2]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个乐于助人的机器人"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [3]:
memory = ConversationBufferMemory(return_messages=True)

In [4]:
memory.load_memory_variables({})

{'history': []}

- 增加一条链
- Add a chain

In [5]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [6]:
inputs = {"input": "你好我是tomie"}
response = chain.invoke(inputs)
response

AIMessage(content='你好，Tomie！有什么问题我可以帮助你解决吗？', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 30, 'total_tokens': 54, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f696101c-3398-49f3-bdb7-8362c370c1eb-0')

In [7]:
#保存记忆
memory.save_context(inputs, {"output": response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='你好我是tomie'),
  AIMessage(content='你好，Tomie！有什么问题我可以帮助你解决吗？')]}

In [8]:
inputs = {"input": "我叫什么名字?"}
response = chain.invoke(inputs)
response

AIMessage(content='您的名字是Tomie。您之前也告诉我了。有什么其他问题我可以帮助您解决吗？', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 71, 'total_tokens': 108, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e51df6b9-7156-4e0b-8b22-77b5c5553459-0')

- 使用Redis来实现长时记忆
**注意：此处运行需要启动redis服务**
- Using Redis for Long-Term Memory
**Note: the redis service needs to be started to run here**

In [13]:
! pip install redis

  Obtaining dependency information for redis from https://files.pythonhosted.org/packages/c5/d1/19a9c76811757684a0f74adc25765c8a901d67f9f6472ac9d57c844a23c8/redis-5.0.8-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/255.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/255.6 kB ? eta -:--:--
   ------ -------------------------------- 41.0/255.6 kB 487.6 kB/s eta 0:00:01
   ------------------- -------------------- 122.9/255.6 kB 1.0 MB/s eta 0:00:01
   ----------------------------------- ---- 225.3/255.6 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 255.6/255.6 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from typing import Optional

from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个擅长{ability}的助手"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI(model="gpt-4-1106-preview",temperature=0)

In [11]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    #使用redis存储聊天记录
    lambda session_id: RedisChatMessageHistory(session_id, url="redis://localhost:6379/0"),
    input_messages_key="question",
    history_messages_key="history",
)

In [12]:
#每次调用都会保存聊天记录，需要有对应的session_id
chain_with_history.invoke(
    {"ability": "历史", "question": "中国建都时间最长的城市是哪个?"},
    config={"configurable": {"session_id": "tomiezhang"}},
)

ImportError: Could not import redis python package. Please install it with `pip install redis`.

In [ ]:
chain_with_history.invoke(
    {"ability": "历史", "question": "它有多少年建都历史？"},
    config={"configurable": {"session_id": "tomiezhang"}},
)